# init

In [3]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
from shapely.geometry import mapping
import matplotlib.pyplot as plt
from pathlib import Path
from rasterio.transform import from_origin
import json
import richdem as rd

In [51]:
#hillshade function from geoexamples.blogspot.com
# Source: http://geoexamples.blogspot.com.br/2014/03/shaded-relief-images-using-gdal-python.html
def hillshade(array, azimuth, angle_altitude):

    x, y = np.gradient(array)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    azimuthrad = azimuth*np.pi / 180.
    altituderad = angle_altitude*np.pi / 180.


    shaded = np.sin(altituderad) * np.sin(slope) \
     + np.cos(altituderad) * np.cos(slope) \
     * np.cos(azimuthrad - aspect)
    return 255*(shaded + 1)/2

In [ ]:
# slope function 
def slope(array):
    x, y = np.gradient(array)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
    return slope

# input

In [6]:
#input a raster file
raster_path = Path("D:\\Dropbox\\x\\PostDoc\\23 10 LMU\\13 exp_cases\\24 01 Urban SSP MAN MUM JAK\\GIS_temp\\")
raster_file = 'MUM_URB_urban_evolution_EOC_WUF.tif'
in_raster = rasterio.open(raster_path/raster_file)

#get the values to extract
values_list = [1985, 2000, 2015]

In [52]:
extracted_raster_list = extract_rasters(in_raster.read(1), values_list)

In [53]:
extracted_raster_list

[array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [55]:
out_path = Path(r'D:\\Dropbox\\x\\PostDoc\\23 10 LMU\\13 exp_cases\\24 01 Urban SSP MAN MUM JAK\\GIS_temp\\')

for n in range(len(values_list)):
    out_file = 'MUM_URB_urbanisation_' + str(values_list[n]) + '_EOC_WUF.tif'
    out_url = out_path/out_file
    print(n, out_url, extracted_raster_list[n].astype(rasterio.uint32))
    
    with rasterio.open(out_url, "w", driver='GTiff',
                    height = in_raster.shape[0], width=in_raster.shape[1],
                    count=1, dtype=list(in_raster.dtypes)[0],
                    crs=in_raster.crs, transform = in_raster.transform) as dest:
        dest.write(extracted_raster_list[n].astype(rasterio.uint32), indexes=1)

0 D:\Dropbox\x\PostDoc\23 10 LMU\13 exp_cases\24 01 Urban SSP MAN MUM JAK\GIS_temp\MUM_URB_urbanisation_1985_EOC_WUF.tif [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
1 D:\Dropbox\x\PostDoc\23 10 LMU\13 exp_cases\24 01 Urban SSP MAN MUM JAK\GIS_temp\MUM_URB_urbanisation_2000_EOC_WUF.tif [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
2 D:\Dropbox\x\PostDoc\23 10 LMU\13 exp_cases\24 01 Urban SSP MAN MUM JAK\GIS_temp\MUM_URB_urbanisation_2015_EOC_WUF.tif [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
